# Постановка задачи

## Замер времени

In [1]:
import time

In [2]:
def some_long_operation():
    time.sleep(3)

хотим логировать время работы этой функции

In [3]:
def some_long_operation():
    start_time = time.time()
    
    time.sleep(3)
    
    end_time = time.time()
    print('duration is', end_time - start_time)
    
some_long_operation()

duration is 3.0041282176971436


## Хотим замерять другую функцию

In [4]:
def another_long_operation():
    start_time = time.time()
    
    time.sleep(1)
    
    end_time = time.time()
    print('duration is', end_time - start_time)
    
another_long_operation()

duration is 1.0015640258789062


# DRY

Декомпозируем

In [5]:
def timeit(func, *args, **kwargs):
    start_time = time.time()
    
    func(*args, **kwargs)
    
    end_time = time.time()
    print('duration is', end_time - start_time)
    
def some_long_operation(sleep_for):
    time.sleep(sleep_for)

In [6]:
timeit(some_long_operation, .1)
timeit(some_long_operation, .2)
timeit(some_long_operation, .5)
timeit(some_long_operation, 1)

duration is 0.10080528259277344
duration is 0.20143747329711914
duration is 0.5006773471832275
duration is 1.0263147354125977


Не хотим писать timeit все время -- как сделать?

In [7]:
def timeit(func, *args, **kwargs):
    def wrapper():
        start_time = time.time()

        result = func(*args, **kwargs)

        end_time = time.time()
        print('duration is', end_time - start_time)
        
        return result
    return wrapper

def some_long_operation(sleep_for):
    time.sleep(sleep_for)
    return sleep_for
    
some_long_operation = timeit(some_long_operation, .1) # нужно сразу передать аргументы, неудобно!

some_long_operation()

duration is 0.11095952987670898


0.1

переделываем

In [8]:
def timeit(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()

        result = func(*args, **kwargs)

        end_time = time.time()
        print('duration is', end_time - start_time)
        
        return result
    return wrapper

def some_long_operation(sleep_for):
    time.sleep(sleep_for)
    
some_long_operation = timeit(some_long_operation)

some_long_operation(.1)
some_long_operation(.2)

duration is 0.1761929988861084
duration is 0.20540904998779297


# типичный шаблон..

In [92]:
def function(...):
    ...

function = make_wrapper(function)

SyntaxError: invalid syntax (3451782019.py, line 1)

## настолько типичный, что даже сделали специальный синтаксис

In [10]:
@make_wrapper
def function():
    ...

NameError: name 'make_wrapper' is not defined

In [ ]:
def function(...):
    ...

function = make_wrapper(function)

In [11]:
def timeit(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()

        result = func(*args, **kwargs)

        end_time = time.time()
        print('duration is', end_time - start_time)
        
        return result
    return wrapper

@timeit
def some_long_operation(sleep_for):
    time.sleep(sleep_for)

some_long_operation(.1)
some_long_operation(.2)

duration is 0.10639071464538574
duration is 0.21231842041015625


## Еще пример

In [12]:
def with_bold(func):
    def wrapper(*args, **kwargs):
        return '<b>' + func(*args, **kwargs) + '</b>'
    return wrapper

def with_italic(func):
    def wrapper(*args, **kwargs):
        return '<i>' + func(*args, **kwargs) + '</i>'
    return wrapper

def make_greeting(username):
    return f'hello {username}'

make_greeting = with_bold(with_italic(make_greeting))
make_greeting('dima')

'<b><i>hello dima</i></b>'

In [13]:
@with_bold
@with_italic
def make_greeting(username):
    return f'hello {username}'

make_greeting('dima')

'<b><i>hello dima</i></b>'

## [warning] порядок применения

In [93]:
@deco1
@deco2
@deco3  # оборачиваем в порядке луковой кожуры
def func():
    ...
    
func = deco3(func)
func = deco2(func)  
func = deco1(func)

NameError: name 'deco1' is not defined

In [17]:
class Foo:
    def __call__(self, *args, **kwargs):
        print(locals())

In [18]:
foo = Foo()

In [20]:
foo(1, b=2)

{'self': <__main__.Foo object at 0x7f47e83c0580>, 'args': (1,), 'kwargs': {'b': 2}}


# Декоратор -- просто callable от одного аргумента
## декоратор через функцию

In [21]:
def timeit(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()

        result = func(*args, **kwargs)

        end_time = time.time()
        print('duration is', end_time - start_time)
        
        return result
    return wrapper

@timeit
def wait_for(seconds):
    time.sleep(seconds)

wait_for(1)

duration is 1.00252103805542


## декоратор через класс

`timeit(func)` должен вернуть что-то, что можно вызвать как функцию

In [27]:
class timeit:
    def __init__(self, func):  # timeit(func)
        self._func = func
        
    def __call__(self, *args, **kwargs):
        start_time = time.time()

        result = self._func(*args, **kwargs)

        end_time = time.time()
        print('duration is', end_time - start_time)
        
        return result

@timeit
def wait_for(seconds):
    time.sleep(seconds)

wait_for(1) 
wait_for(2)

duration is 1.0901024341583252
duration is 2.003375768661499


In [ ]:
from functools import lru_cache

In [ ]:
wait_for._func

# Декоратор класса

In [31]:
def set_foo(cls):
    cls.foo = 'bar'
    return cls
    
@set_foo
class Foo:
    ...
      
dir(Foo)  # в коде ошибка

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'foo']

In [32]:
def set_foo(cls):
    print('set_foo decorator called')
    cls.foo = 'bar'
    return cls

def add_foo_func(cls):
    print('set add_foo_func decorator called')
    def foo_func(self):
        print(self.foo)
    cls.foo_func = foo_func
    return cls
    
@add_foo_func
@set_foo
class Foo:
    ...

dir(Foo)

set_foo decorator called
set add_foo_func decorator called


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'foo',
 'foo_func']

In [33]:
print(Foo.foo)

bar


In [34]:
Foo().foo_func()

bar


## dataclasses

[посмотрите доку!](https://docs.python.org/3/library/dataclasses.html#module-contents)

In [53]:
class User:
    def __init__(self, name: str, surname: str, grade: int = 0):
        self.name = name
        self.surname = surname
        self.grade = grade
    
    def __str__(self) -> 'str':
        return f'** user with name {self.name}'
    
    def __repr__(self) -> 'str':
        return f'User(name={self.name!r}, surname={self.surname!r}, grade={self.grade!r})'
    
#     def __eq__(self, other) -> bool

        
# print(str(User('name', 'surname', 10)))
# print(repr(User('name', 'surname', 10)))
# print(User('name', 'surname', 10) == User('name', 'surname', 10))

In [55]:
kwargs = dict(
    name='dima',
    surname='none',
    grade=10,
)
eval(
    repr(User(**kwargs))
) == User(**kwargs)

False

In [42]:
str(User('dime', None))

'** user with name dime'

In [56]:
import dataclasses
@dataclasses.dataclass
class User:
    name: str
    surname: str
    grade: int = 0
        
print(str(User('name', 'surname', 10)))
print(repr(User('name', 'surname', 10)))
print(User('name', 'surname', 10) == User('name', 'surname', 10))

User(name='name', surname='surname', grade=10)
User(name='name', surname='surname', grade=10)
True


# Декораторы с аргументами

```python
@deco(arg=1)
def func(*args, **kwargs):
    ...
```

равносильно

```python
def func(*args, **kwargs):
    ...

func = deco(arg=1)(func)
```    

# staticmethod, classmethod

## Зачем нужны?

In [57]:
class TimeUtils:
    def now(self):
        print(time.time())

In [59]:
TimeUtils().now()

1646927649.1941116


In [60]:
TimeUtils.now()

TypeError: now() missing 1 required positional argument: 'self'

### now не зависит от self, хотим вызывать через класс

In [61]:
class TimeUtils:
    def now():
        print(time.time())

TimeUtils.now()

1646927668.8145907


In [62]:
# вроде работает!

In [64]:
TimeUtils().now()

TypeError: now() takes 0 positional arguments but 1 was given

### что происходит?

In [68]:
class Debugger:
    def debug(*args):
        print(locals())

In [69]:
Debugger.debug()
Debugger().debug()

{'args': ()}
{'args': (<__main__.Debugger object at 0x7f47cf03d460>,)}


In [71]:
print(Debugger.debug)
print(Debugger().debug)

<function Debugger.debug at 0x7f47cecb8040>
<bound method Debugger.debug of <__main__.Debugger object at 0x7f47cefce4f0>>


tl;dr -- питон "связывает" метод к инстансу класса, если идет доступ через инстанс класса. Что проихсодит на самом деле, обсудим позже

## staticmethod to the rescue!

In [77]:
class Debugger:
    def method(*args):
        print(locals())
    
    @staticmethod
    def method_with_static(*args):
        print(locals())
        
#     @classmethod
#     def method_with_class(*args):
#         print(locals())

    @classmethod
    def method_with_class(cls):
        cls.method_with_static
        
    
Debugger.method()
Debugger().method()

Debugger.method_with_static()
Debugger().method_with_static()

Debugger.method_with_class()
Debugger().method_with_class()

{'args': ()}
{'args': (<__main__.Debugger object at 0x7f47ced39e20>,)}
{'args': ()}
{'args': ()}
{'args': (<class '__main__.Debugger'>,)}
{'args': (<class '__main__.Debugger'>,)}


In [80]:
class Foo:
    @staticmethod
    def bunc():
        print('bunc called')
    
    @classmethod
    def func(cls):
        print(cls)

class Boo(Foo):
    pass

Foo.func()
Boo.func()

Foo.bunc()
Boo.bunc()

<class '__main__.Foo'>
<class '__main__.Boo'>
bunc called
bunc called


In [220]:
Debugger.__dict__

mappingproxy({'__module__': '__main__',
              'method': <function __main__.Debugger.method(*args)>,
              'method_with_static': <staticmethod at 0x7fe7800e75e0>,
              'method_with_class': <classmethod at 0x7fe7800e7790>,
              '__dict__': <attribute '__dict__' of 'Debugger' objects>,
              '__weakref__': <attribute '__weakref__' of 'Debugger' objects>,
              '__doc__': None})

### и это тоже обычный декоратор!

In [221]:
staticmethod, type(staticmethod)

(staticmethod, type)

In [222]:
classmethod, type(classmethod)

(classmethod, type)

In [ ]:
class Debugger:
    def method(*args):
        print(locals())
    
    def method_with_static(*args):
        print(locals())
    method_with_static = staticmethod(method_with_static)
    
    def method_with_class(*args):
        print(locals())
    method_with_class = classmethod(method_with_class)

## Вопросы
- В каких случаях надо использовать staticmethod, а в каких classmethod?
- Может ли classmethod вызвать staticmethod? А наоборот?

# Бочку с дегтем этому господину!

In [82]:
def super_func(arg: int) -> str:
    '''
    this super_func will return a string!
    '''
    return str(arg)

In [83]:
help(super_func)

Help on function super_func in module __main__:

super_func(arg: int) -> str
    this super_func will return a string!



In [240]:
super_func.__name__

'super_func'

In [86]:
def decorator(func):
    def wrapper(*args, **kwargs):
        print('starting')
        return func(*args, **kwargs)
    return wrapper

@decorator
def super_func(arg: int) -> str:
    '''
    this super_func will return a string!
    '''
    return str(arg)

In [87]:
super_func(1)

starting


'1'

In [88]:
help(super_func)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



In [243]:
super_func.__name__

'wrapper'

In [91]:
import functools

In [92]:
def decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('starting')
        return func(*args, **kwargs)
    return wrapper

@decorator
def super_func(arg: int) -> str:
    '''
    this super_func will return a string!
    '''
    return str(arg)

In [93]:
help(super_func)

Help on function super_func in module __main__:

super_func(arg: int) -> str
    this super_func will return a string!



## можно ли так сделать с классом?

In [ ]:
def super_func(arg: int) -> str:
    '''
    this super_func will return a string!
    '''
    return str(arg)

In [94]:
class decorator:
    def __init__(self, func):  # timeit(func)
        self._func = func
        
    def __call__(self, *args, **kwargs):
        print('starting')
        return self._func(*args, **kwargs)

@decorator
def super_func(arg: int) -> str:
    '''
    this super_func will return a string!
    '''
    return str(arg)

In [95]:
help(super_func)

Help on decorator in module __main__ object:

class decorator(builtins.object)
 |  decorator(func)
 |  
 |  Methods defined here:
 |  
 |  __call__(self, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __init__(self, func)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [96]:
# @functools.wraps использует под капотом functools.update_wrapper

In [105]:
class decorator:
    def __init__(self, func):  # timeit(func)
        self._func = func
        functools.update_wrapper(self, func)
        
    def __call__(self, *args, **kwargs):
        print('starting')
        return self._func(*args, **kwargs)

@decorator
def super_func(arg: int) -> str:
    '''
    this super_func will return a string!
    '''
    return str(arg)

In [107]:
super_func.__doc__

'\n    this super_func will return a string!\n    '

In [108]:
super_func.__name__

'super_func'

In [109]:
help(super_func)

Help on decorator in module __main__ object:

super_func = class decorator(builtins.object)
 |  super_func(func)
 |  
 |  Methods defined here:
 |  
 |  __call__(self, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __init__(self, func)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [261]:
super_func.__name__

'super_func'

In [262]:
super_func.__doc__

'\n    this super_func will return a string!\n    '

# [functools](https://docs.python.org/3/library/functools.html)